In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor as rfr, AdaBoostRegressor as abr, GradientBoostingRegressor as gbr
from sklearn.linear_model import LinearRegression as lr
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error as mse

import xgboost as xgb
import lightgbm as lgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df_orig = pd.read_csv("/kaggle/input/black-friday-anallytics-vidhya/train_oSwQCTC/train.csv")
test_df_orig = pd.read_csv("/kaggle/input/black-friday-anallytics-vidhya/test_HujdGe7/test.csv")
submit = pd.read_csv("/kaggle/input/black-friday-anallytics-vidhya/sample_submission_V9Inaty.csv")

train_df = train_df_orig.copy()
test_df = test_df_orig.copy()
# train_df["Occupation"] = train_df["Occupation"].astype('O')
# train_df["Marital_Status"] = train_df["Marital_Status"].astype('O')

In [ ]:
train_df_orig.sample(10)

In [ ]:
train_df.describe()

1. Looking at purchase mean, std, min , max and percentiles especially, it seems to be a normal dist (or close to it)
2. Marital_Status depicts that only 40% of total population is married
3. Count values of Product_Category_2 and Product_Category_3 shows there are missing values. (May be because not many people buy those category products in general).

In [ ]:
h = train_df_orig[train_df_orig.columns[2:]].hist(figsize=(10,8))

In [ ]:
h = sns.distplot(train_df_orig.Purchase, bins=10)

In [ ]:
corr = train_df.corr()
corr["Purchase"]

In [ ]:
sns.heatmap(corr, annot=True)

Correlations with Purchase
* Good negative correlation with Product_Category_1, Product_Category_2 and 3 as well. Why?
* A bit good positive correlation with occupation

Also all product categories are highly correlated to each other.

In [ ]:
num_cols = []
cat_cols = []

for col in train_df.columns[2:-1]:
    if train_df[col].dtype == 'int64' or train_df[col].dtype == 'float64':
        num_cols.append(col)
    else:
        cat_cols.append(col)
cat_cols

In [ ]:
for col in train_df.columns:
    if train_df[col].nunique()<30:
        print({col:train_df[col].unique()})
    else:
        print({col:train_df[col].nunique()})

In [ ]:
for col in cat_cols+num_cols:
    sns.barplot(data=train_df, x=col, y="Purchase")
    plt.show()

In [ ]:
# train_df["Gender"] = train_df["Gender"].map({'F':0.0001, 'G':0.9999})
# train_df["Age"] = train_df["Age"].map({'51-55':1, ''})

In [ ]:
for col in num_cols:
    sns.scatterplot(data=train_df_orig, x=col, y="Purchase")
    plt.show()

Let us just encode our categorical features and replace nan values with -999

In [ ]:
train_df["Product_ID"] = train_df["Product_ID"].str[1:].astype('int')
test_df["Product_ID"] = test_df["Product_ID"].str[1:].astype('int')

In [ ]:
indices = train_df.pivot_table(["Purchase"], "Product_ID").index
values = train_df.pivot_table(["Purchase"], "Product_ID").values
train_df["purchase_avg_orderby_pro_id"] = np.nan
test_df["purchase_avg_orderby_pro_id"] = np.nan
for i,v in enumerate(values):
    train_df.loc[(train_df["Product_ID"]==indices[i]), "purchase_avg_orderby_pro_id"] = v
    test_df.loc[(test_df["Product_ID"]==indices[i]), "purchase_avg_orderby_pro_id"] = v

In [ ]:
indices = train_df.pivot_table(["Purchase"], "User_ID").index
values = train_df.pivot_table(["Purchase"], "User_ID").values
train_df["purchase_avg_orderby_user_id"] = np.nan
test_df["purchase_avg_orderby_user_id"] = np.nan
for i,v in enumerate(values):
    train_df.loc[(train_df["User_ID"]==indices[i]), "purchase_avg_orderby_user_id"] = v
    test_df.loc[(test_df["User_ID"]==indices[i]), "purchase_avg_orderby_user_id"] = v

In [ ]:
# for col in cat_cols:
#     col_order_by_purchase = train_df.pivot_table(['Purchase'], col).sort_values(by="Purchase").index.to_list()
#     train_df[col] = train_df[col].apply(lambda x:col_order_by_purchase.index(x)+1)
#     test_df[col] = test_df[col].apply(lambda x:col_order_by_purchase.index(x)+1)

In [ ]:
for col in cat_cols+["Occupation", "Marital_Status"]:
    _ = train_df.groupby(col).describe()["Purchase"]
    df = (_["mean"]/_["count"])
    train_df[col] = train_df[col].apply(lambda x: df[x])
    test_df[col] = test_df[col].apply(lambda x:df[x])

In [ ]:
# train_df['Gender'] = train_df['Gender'].replace({'F':0.0001, 'M':0.9999})
# test_df['Gender'] = test_df['Gender'].replace({'F':0.0001, 'M':0.9999})

# dum_cols = ['Age', 'City_Category', 'Stay_In_Current_City_Years']
# train_df = pd.get_dummies(data=train_df, columns=dum_cols, drop_first=True)
# test_df = pd.get_dummies(data=test_df, columns=dum_cols, drop_first=True)

# train_df['']
# train_df['Age'] = train_df['Age'].replace({'0-17':1, '55+', '26-35':3, '46-50', '51-55', '36-45', '18-25':2})



# for col in cat_cols:
#     le = LabelEncoder()
#     train_df[col] = le.fit_transform(train_df[col])
#     test_df[col] = le.fit_transform(test_df[col])

In [ ]:
train_df = train_df.fillna(-999)
test_df = test_df.fillna(-999)

In [ ]:
train_df.head(2)

In [ ]:
train_df.columns

In [ ]:
features = ['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3',
       'purchase_avg_orderby_pro_id', 'purchase_avg_orderby_user_id']

In [ ]:
cat_cols

In [ ]:
corr = train_df[features+['Purchase']].corr()
corr["Purchase"]

In [ ]:
X = train_df[features]
y = train_df["Purchase"]

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# models = [rfr(), gbr(), abr(), lr()]

# for model in models:
#     model.fit(train_X, train_y)
#     pred = model.predict(val_X)
#     print(np.sqrt(mse(val_y, pred)))

In [ ]:
# kf = KFold()
# model = rfr()
# X_ = train_df[features].values
# for train_idx, val_idx in kf.split(X_):
#     X_train, y_train = X_[train_idx], y[train_idx]
#     X_val, y_val = X_[val_idx], y[val_idx]
#     model.fit(X_train, y_train)
#     pred = model.predict(X_val)
#     score = np.sqrt(mse(y_val, pred))
#     print(score)

In [ ]:
# test_pred = model.predict(test_df[features])
# submit["Purchase"] = test_pred
# submit.to_csv('black_friday_sales_prediction_gbr.csv')

In [ ]:
parameters = {
        'eta':0.1,
        'min_child_weight':10,
        'max_depth':8,
        'subsample':0.7,
        'colsample_bytree':0.7,
        'scale_pos_weight':0.8
    }

dtrain = xgb.DMatrix(X, y)
dval = xgb.DMatrix(val_X, val_y)

xgbr = xgb.train(params=parameters,
              dtrain=dtrain,
              num_boost_round=1470,
              evals=[(dval, "Test")],
              verbose_eval=100,
              early_stopping_rounds=10)

print(np.sqrt(mse(xgbr.predict(xgb.DMatrix(val_X)), val_y)))

In [ ]:
test_pred = xgbr.predict(xgb.DMatrix(test_df[features]))
submit["Purchase"] = test_pred
submit.to_csv('black_friday_sales_prediction_xgbr.csv')

In [ ]:
submit.head(2)

In [ ]:
xgb.plot_importance(xgbr)

In [ ]:
xgbr = xgb.XGBRegressor()
xgbr.fit(train_X, train_y)
print(np.sqrt(mse(xgbr.predict((val_X)), val_y)))

In [ ]:
xgbr.get_params()

In [ ]:
# from sklearn.model_selection import GridSearchCV

# param_test1 = {
#  'max_depth':range(3,10,2),
#  'min_child_weight':range(1,6,2)
# }
# gsearch1 = GridSearchCV(estimator = xgb.XGBRegressor(objective= 'reg:squarederror',base_score= 0.5,
#                                                  booster= 'gbtree',colsample_bylevel= 1,
#                                                  colsample_bynode= 1,colsample_bytree= 1,gamma= 0,
#                                                  importance_type= 'gain',learning_rate= 0.300000012,
#                                                  max_delta_step= 0,n_estimators= 100,n_jobs= 0,
#                                                  num_parallel_tree= 1,random_state= 0,reg_alpha= 0,
#                                                  reg_lambda= 1,scale_pos_weight= 1,subsample= 1,
#                                                  tree_method= 'exact',verbosity= None), 
#                         param_grid = param_test1, cv=5)
# gsearch1.fit(train_X,train_y)
# gsearch1.best_params_, np.sqrt(mse(gsearch1.predict((val_X)), val_y))

In [ ]:
TPOTRegressor?

In [ ]:
from tpot import TPOTRegressor

tpr = TPOTRegressor(generations=50, population_size=5, early_stop=20, n_jobs=-1, verbosity=3)
tpr.fit(X, y)

In [ ]:
print(np.sqrt(mse(tpr.predict(val_X), val_y)))

In [ ]:
tpr.fitted_pipeline_

In [ ]:
test_pred = tpr.predict(test_df[features])
submit["Purchase"] = test_pred
submit.to_csv('black_friday_sales_prediction_tpr.csv')